In [45]:
import pandas as pd
import numpy as np
import os
import pickle

In [46]:
from lxml import etree
import xml.etree.cElementTree as ET
import distance

In [76]:
import re
def to_normal(a1):
    a1 = re.sub(r'\d+', '', a1)
    a1 = re.sub(r'[^\w\s]','',a1)
    a1 = re.sub('\n', '', a1)
    a1 = a1.strip()
    a1 = a1.lower()
    
    #print a1
    return a1

def find_match(a1,a2):    
    a1 = to_normal(a1)
    dis = 1
    paper = None
    for elem in a2:
        
        score = distance.nlevenshtein(a1, elem)
        #print 'comparing: \n'+ a1 +'\n'+ elem
        #print score
        if score<dis:
            paper = elem
            dis = score
    if dis<0.1:
        return paper

    #for elem in a2:
        #elem = to_normal(elem)   
    #print fuzz.token_set_ratio(a1,a2)
    #print process.extractOne(a1, [a2])
        #return process.extractOne(a1, a2)[0]
    return False

In [69]:
#reading data
network = pd.read_csv("network.csv")

#Creating incite and outcite dictionary
incite = {}
outcite = {}
for row in network.iterrows():
    if(type(row[1][1])!=str):
        incite[row[1][0]] = []
    else:
        incite[row[1][0]] = row[1][1].split(',')
    if(type(row[1][2])!=str):
        outcite[row[1][0]] = []
    else:
        outcite[row[1][0]] = row[1][2].split(',')

paper_directory = "../2014/papers_text"

#paper_array = names of all papers in dataset ordered lexically
paper_array = []
for filename in os.listdir(paper_directory):
    if filename.endswith(".txt"): 
        #print(os.path.join(directory, filename))
        paper_array.append(filename[:-4])
paper_array = sorted(paper_array)

In [108]:
outcite_1 = {}
incite_1 = {}
for paper_id in paper_array[:50]:
    incite_1[paper_id] = []
    outcite_1[paper_id] = []

paper_ids=open("../2014/paper_ids.txt","r")
dict_id={}
for line in paper_ids:
    key=line[:8]
    value=to_normal(line[9:-5].strip())
    dict_id[key]=value

In [109]:
for curr_paper in outcite_1.keys():
    
    tree = ET.ElementTree(file= '../data/xml/A/A00/'+curr_paper+'-parscit.130908'+'.xml')
    root = tree.getroot()

    heading = []
    body = []
    count = -1
    for elem in root[0][0]:
        count+=1
        if elem.tag == 'sectionHeader' or elem.tag =='subsectionHeader':
            heading.append([count, elem.text])
        if elem.tag == 'bodyText':
            body.append([count, elem.text])
        
#print heading[:9]
#print body[1][1]

    cit_head_map = {}
    for elem in tree.iter(tag='citation'):
        heads_ind = []
        citstr = []
        title = []
        body_ind = []
        for child in elem:
            if child.tag == 'title':
                title = child.text       
            if child.tag == 'contexts':            
                citstr = child[0].attrib['citStr']
            
        if (citstr == []) or (title == []):
            continue
        else:
        #print 'title: '+ title
        #print 'citStr: '+ citstr
    
            for entry in body:
                if citstr in entry[1]:
                    body_ind.append(entry[0])
            
            if len(body_ind)!=0:
                for ind in body_ind:
                    #print 'body: '+ str(ind)
                    sect_ind = -1
                    for entry in heading:
                        if entry[0]<ind and entry[0]>sect_ind:
                            sect_ind = entry[0]
                    #print sect_ind
                    sect = root[0][0][sect_ind].text
                    sect = to_normal(sect)
                    #print 'sect:'+ sect
                    cit_head_map[to_normal(title)]=sect.strip()
                
            else:
                continue

    for entry in outcite[curr_paper]:
        #print entry
        matched = find_match(dict_id[entry],cit_head_map.keys())
        if matched:
            #print 'match_found !'
            outcite_1[curr_paper].append([entry,cit_head_map[matched]])                    

In [ ]:
with open("dict.txt", "wb") as dict_file:
    pickle.dump(outcite_1, dict_file)

In [ ]:
#result for A00-1001 ---to--- A00-1010  for analyses:

In [103]:
for key in outcite_1.keys():
    print key
    print outcite[key]
    

A00-1010
['H94-1010', 'P96-1008', 'W00-0303']
A00-1009
['A92-1006', 'A97-1037', 'A97-1039', 'C92-3158', 'J85-1003', 'J94-4004']
A00-1008
['A00-2041', 'J86-3001', 'P98-1059']
A00-1001
['J82-3002']
A00-1003
['P98-1066', 'P99-1027', 'P99-1029']
A00-1002
['C90-3057', 'P98-1080']
A00-1005
['J86-3001', 'J99-3003', 'P96-1009']
A00-1004
['A00-1019', 'J93-1006', 'J93-2003', 'P91-1022', 'P91-1023', 'P93-1002', 'P94-1012']
A00-1007
['J96-2004']
A00-1006
['C96-1070', 'P98-2131', 'P98-2185', 'W97-0403']


In [105]:
outcite_1

{'A00-1001': [['J82-3002', 'previous efforts chatprat and hsql']],
 'A00-1002': [],
 'A00-1003': [['P98-1066', 'evaluation']],
 'A00-1004': [['A00-1019', 'introduction'],
  ['P91-1022', 'parallel text alignment'],
  ['P93-1002', 'parallel text alignment'],
  ['P94-1012', 'parallel text alignment']],
 'A00-1005': [['J99-3003', 'previous work']],
 'A00-1006': [['C96-1070', 'transfer driven machinetranslation'],
  ['W97-0403', 'introduction']],
 'A00-1007': [],
 'A00-1008': [],
 'A00-1009': [['A92-1006',
   'history of the framework and comparisonwith other systems'],
  ['A97-1037', 'history of the framework and comparisonwith other systems'],
  ['A97-1039', 'history of the framework and comparisonwith other systems'],
  ['C92-3158', 'history of the framework and comparisonwith other systems'],
  ['J85-1003', 'introduction'],
  ['J94-4004', 'the frameworks linguistic resources']],
 'A00-1010': []}

In [101]:
# to load the dict_file use pickle:

#import pickle
#with open("dict.txt", "rb") as dict_file:
#    out_1 = pickle.load(dict_file)

